# Intersection CSV to GeoJSON Converstion Script

Input list of intersections as CSV file in format: ID, Name1, Name2, Latitude, Longitude

Where Name 1 could be north/south street name and Name 2 could be east/west street name

### CSV Input Example:
2,California Blvd,Ygnacio Valley Rd,37.904976, -122.065751


Outputs GeoJSON feature files from input data and adds elevation in feet when available.

### GeoJSON Output Example:
{
    "geometry": {
        "coordinates": [
            -122.065751,
            37.904976,
            156.12
        ],
        "type": "Point"
    },
    "id": "02",
    "properties": {
        "name": "California Blvd & Ygnacio Valley Rd"
    },

In [ ]:
#imports
from geojson import Point, Feature, FeatureCollection, dump
import csv

import requests
import urllib

import os, sys

In [ ]:
#Variables

features = []
input_filename = "../WalnutCreekIntersections.csv"
output_filename = "../WalnutCreekIntersections-Mar2022.geojson"
url = r'https://nationalmap.gov/epqs/pqs.php?'


In [ ]:
def elevation_of_location(lat, lon):
    """Query service using lat, lon. Add the elevation values as a new column."""
    # define rest query params
    params = {
        'output': 'json',
        'x': lon,
        'y': lat,
        'units': 'Feet' #'Meters' is also available
    }
    # format query string and return query value
    result = requests.get((url + urllib.parse.urlencode(params)))
    if result is not None:
        return result.json()['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation']
    else: 
        return None
        

In [ ]:
def read_file():
    "read in csv file and output geojson file"
    with open(input_filename, "r") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=",")
        line_count = 0
        for row in csv_reader:
            lat = float(row[3])
            lon = float(row[4])
            ele = elevation_of_location(lat,lon)
            if ele is not None:
                features.append(Feature(geometry=Point( ( lon, lat, ele )), properties={"name": row[1] +" & "+ row[2]}, id=row[0] ))
                
            else:
                features.append(Feature(geometry=Point( ( lon, lat)), properties={"name": row[1] +" & "+ row[2]}, id=row[0] ))
                
            line_count += 1
            print(f'Processing line #{line_count}')
            
        feature_collection = FeatureCollection(features)
        with open(output_filename, 'w') as f:
        	dump(feature_collection, f, sort_keys=True, indent=4, separators=(',', ': '))
        print(f"Processed {line_count} intersections and saved as {output_filename}")
        



In [ ]:
read_file()

#python3 -m json.tool output_filename.geojson > output_filenameV2.geojson



